In [ ]:
# Azure Databricks config
app_id = "APP_ID"
client_secret = "CLIENT_SECRET"
tenant = "TENANT"

configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": app_id,
       "fs.azure.account.oauth2.client.secret": client_secret,
       "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant}/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

In [ ]:
# test that the mounting works, this is the storage container root
import os
os.listdir('/dbfs/mnt/data')

Out[2]: ['collections',
 'dataset',
 'dataset.csv',
 'dataset_clean.csv',
 'dataset_mirlclub.csv',
 'scraped_images']

In [ ]:
########################################################################################
# process kaggle dataset - select columns we are interested in and order the columns nicely
########################################################################################

In [ ]:
kaggleDF = spark.read.options(header=True).csv('/mnt/data/dataset.csv')
kaggleDF.printSchema()
kaggleDF.show(truncate=False)

root
-- title: string (nullable = true)
-- name: string (nullable = true)
-- creator: string (nullable = true)
-- art_series: string (nullable = true)
-- price: string (nullable = true)
-- symbol: string (nullable = true)
-- type: string (nullable = true)
-- likes: string (nullable = true)
-- nsfw: string (nullable = true)
-- tokens: string (nullable = true)
-- year: string (nullable = true)
-- rights: string (nullable = true)
-- royalty: string (nullable = true)
-- cid: string (nullable = true)
-- path: string (nullable = true)

+----------------------------+--------------------------+--------------+--------------------------------------------------+-----+---------+-----+-----+-----+------+----+------+-------+----------------------------------------------+-------------------------------------------------------------------+
title |name |creator |art_series |price|symbol |type |likes|nsfw |tokens|year|rights|royalty|cid |path |
+----------------------------+--------------------------+--------------+--------------------------------------------------+-----+---------+-----+-----+-----+------+----+------+-------+----------------------------------------------+-------------------------------------------------------------------+
30 min Drawings |Giant Frog |kristyglas |kristyglas_30-min-drawings_giant-frog |50 |SWAP.HIVE|PHOTO|2 |False|30 |2020|1 |0 |QmegjZDN2gyD9A6cG4efj8DwbLLGTW6isDGadNC2yVJHfH|./dataset/image/QmegjZDN2gyD9A6cG4efj8DwbLLGTW6isDGadNC2yVJHfH.jpg |
Experimental Video |Biospecimens |juliakponsford|juliakponsford_experimental-video_biospecimens |500 |SWAP.HIVE|VIDEO|0 |False|1 |2020|1 |0 |QmeN4KuTQwHPWvGDhiopNJziqmf1feGeabDD4Gk7z9DgUL|./dataset/video/QmeN4KuTQwHPWvGDhiopNJziqmf1feGeabDD4Gk7z9DgUL.mp4 |
Sexy Art |long legs |badsexy |badsexy_sexy-art_long-legs |10 |SWAP.HIVE|PHOTO|0 |True |2 |2021|1 |0 |QmaifAjZTJd8jTxNrwSoGWmFpbPYcszMF4ZcZDBwpRR4v4|./dataset/image/QmaifAjZTJd8jTxNrwSoGWmFpbPYcszMF4ZcZDBwpRR4v4.jpeg|
Dream World |A Guide in my Dreams |yoslehz |yoslehz_dream-world_a-guide-in-my-dreams |20 |SWAP.HIVE|PHOTO|1 |False|2 |2020|1 |0 |QmUTicJApSyByFb7sgsNa6cz8MG5RKsHvGLP61gxSF3JsN|./dataset/image/QmUTicJApSyByFb7sgsNa6cz8MG5RKsHvGLP61gxSF3JsN.jpeg|
Dream World |Silent Observer |yoslehz |yoslehz_dream-world_silent-observer |20 |SWAP.HIVE|GIF |0 |False|2 |2020|1 |0 |Qmb8WbXXdorG52sgY4mSyYUjevE65SGa2vhSxdXstfinz1|./dataset/gif/Qmb8WbXXdorG52sgY4mSyYUjevE65SGa2vhSxdXstfinz1.gif |
Dream World |Joy and Chaos |yoslehz |yoslehz_dream-world_joy-and-chaos |20 |SWAP.HIVE|GIF |0 |False|2 |2020|1 |0 |Qmaa8hLjotwF8DXvBfLbhe4wa2S8KNL4CE35EcDtWV1Xhf|./dataset/gif/Qmaa8hLjotwF8DXvBfLbhe4wa2S8KNL4CE35EcDtWV1Xhf.gif |
Dream World |I found colors in my sky |yoslehz |yoslehz_dream-world_i-found-colors-in-my-sky |20 |SWAP.HIVE|PHOTO|0 |False|2 |2020|1 |0 |QmT36BPdeJWNEdUr566z4DYGWxXqrDSeoA7zCHbKw8528w|./dataset/image/QmT36BPdeJWNEdUr566z4DYGWxXqrDSeoA7zCHbKw8528w.png |
nothing is real, all is real|Isn´t she lovely |solymi |solymi_nothing-is-real-all-is-real_isnt-she-lovely|19 |SWAP.HIVE|GIF |2 |True |6 |2021|1 |0 |QmR8p6kKW6cUB1Jc7F2yoLGmgV38pPwVK6eW2zhbuwDrVF|./dataset/gif/QmR8p6kKW6cUB1Jc7F2yoLGmgV38pPwVK6eW2zhbuwDrVF.gif |
trippy_emmeline |Emmeline on LSD |solymi |solymi_trippy-emmeline_emmeline-on-lsd |64 |SWAP.HIVE|GIF |1 |False|3 |2021|1 |0 |QmT8sh5XLP83x1CWEksJJajQ7xKe3MAejPUjke9D8grPbS|./dataset/gif/QmT8sh5XLP83x1CWEksJJajQ7xKe3MAejPUjke9D8grPbS.gif |
experimental |Angela 👼 |elias15g |elias15g_experimental_angela- |35 |SWAP.HIVE|PHOTO|3 |False|5 |2020|1 |0 |QmWQGXNu47pecHGfEatd3ATakphq5zXVrzs5unYYK4VR1c|./dataset/image/QmWQGXNu47pecHGfEatd3ATakphq5zXVrzs5unYYK4VR1c.png |
Tentacle Monster |Tentacle Monster |theia7 |theia7_tentacle-monster_tentacle-monster |68 |SWAP.HIVE|GIF |0 |False|1 |2021|1 |0 |QmfGX5kdXvc77PYsgA4GUkNkyxNamiKB2qndPBYMuvHgqF|./dataset/gif/QmfGX5kdXvc77PYsgA4GUkNkyxNamiKB2qndPBYMuvHgqF.gif |
Crypto Stamps |Hive Crypto Stamp |aleister |aleister_crypto-stamps_hive-crypto-stamp |40 |SWAP.HIVE|GIF |3 |False|10 |202

In [ ]:
kaggleDF = bigDF.withColumnRenamed("cid", "id").filter("path LIKE './dataset/image/%'").select("id", "name", "path", "price")
kaggleDF.show()

+--------------------+--------------------+--------------------+-----+
 id| name| path|price|
+--------------------+--------------------+--------------------+-----+
QmegjZDN2gyD9A6cG...| Giant Frog|./dataset/image/Q...| 50|
QmaifAjZTJd8jTxNr...| long legs|./dataset/image/Q...| 10|
QmUTicJApSyByFb7s...|A Guide in my Dreams|./dataset/image/Q...| 20|
QmT36BPdeJWNEdUr5...|I found colors in...|./dataset/image/Q...| 20|
QmWQGXNu47pecHGfE...| Angela 👼|./dataset/image/Q...| 35|
Qma6eHAWkXhrkHsfC...| MONAPIZZA|./dataset/image/Q...| 9.5|
QmeTAUBQUoqekVQBh...| Darkness|./dataset/image/Q...| 16|
QmUmwLohsrgjHac27...| Poor|./dataset/image/Q...| 15|
QmWSozMKfzNE43WoA...| Abducted |./dataset/image/Q...| 50|
QmfDRZnYUtM96U8Ag...|Tame the inner fu...|./dataset/image/Q...| 30|
QmSuqPZsf9xxcEkdD...| Disturbed|./dataset/image/Q...| 17|
QmbJ5uAYt3PDjEHZN...| Crypto Daemons|./dataset/image/Q...| 11|
QmaWvdqVZQeQXqSfR...| surprised by the...|./dataset/image/Q...| 11|
QmaDVBjsjmpPwHun3...| resurface|./dataset/image/Q...| 11|
QmNiRqeW84jnEbN8w...| Avarice|./dataset/image/Q...| 20|
QmeddHYx6UDmTRRLD...| My body, My choice|./dataset/image/Q...| 25|
QmQWykiSvoYKtTM1S...|A little bit of M...|./dataset/image/Q...| 25|
QmaM1zTWoiYwT5L4B...| Angel Killer|./dataset/image/Q...| 50|
QmU1kCAghsy5DCaWP...| Dream of A Child|./dataset/image/Q...| 50|
QmQ9zjvq63ytyXmRh...| Místicismo|./dataset/image/Q...| 50|
+--------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [ ]:
# saves the DF back to disk (potential bottleneck!)
kaggleDF.toPandas().to_csv("/dbfs/mnt/data/dataset_clean.csv", sep=',', header=True, index=False)

In [ ]:
########################################################################################
# retrieve images for a collection
########################################################################################

In [ ]:
collectionDF = spark.read.json('/mnt/data/collections/mirlclub.json')
collectionDF.printSchema()
collectionDF.show()

root
-- id: long (nullable = true)
-- image: string (nullable = true)
-- name: string (nullable = true)
-- price: double (nullable = true)

+----+--------------------+----------+------+
 id| image| name| price|
+----+--------------------+----------+------+
 91|https://lh3.googl...| MIRL #91|0.0349|
1693|https://lh3.googl...|MIRL #1693| 0.035|
 639|https://lh3.googl...| MIRL #639| 0.035|
 462|https://lh3.googl...| MIRL #462| 0.037|
1545|https://lh3.googl...|MIRL #1545| 0.04|
2104|https://lh3.googl...|MIRL #2104| 0.04|
 178|https://lh3.googl...| MIRL #178| 0.04|
 385|https://lh3.googl...| MIRL #385| 0.045|
1521|https://lh3.googl...|MIRL #1521| 0.045|
1522|https://lh3.googl...|MIRL #1522| 0.045|
 724|https://lh3.googl...| MIRL #724| 0.05|
2207|https://lh3.googl...|MIRL #2207| 0.05|
 628|https://lh3.googl...| MIRL #628| 0.05|
1899|https://lh3.googl...|MIRL #1899| 0.05|
 665|https://lh3.googl...| MIRL #665| 0.055|
2439|https://lh3.googl...|MIRL #2439| 0.06|
1471|https://lh3.googl...|MIRL #1471| 0.06|
1805|https://lh3.googl...|MIRL #1805| 0.065|
 621|https://lh3.googl...| MIRL #621| 0.069|
 703|https://lh3.googl...| MIRL #703| 0.07|
+----+--------------------+----------+------+
only showing top 20 rows

In [ ]:
from urllib.request import urlretrieve
def download_image(collection, name, url):
    urlretrieve(url, f"/dbfs/mnt/data/scraped_images/{collection}-{name}.jpg")

collectionDF.foreach(lambda row: download_image('mirlclub', row['name'], row['image']))

In [ ]:
from pyspark.sql.functions import *
collectionDF = collectionDF.withColumn("id", concat(lit('mirlclub-'), col('id')))
collectionDF = collectionDF.withColumn("path", concat(lit('./scraped_images/mirlclub-'), col('name'), lit('.jpg')))
collectionDF = collectionDF.select('id', 'name', 'path', 'price')
collectionDF.show(truncate=False)

+-------------+----------+----------------------------------------+------+
id |name |path |price |
+-------------+----------+----------------------------------------+------+
mirlclub-91 |MIRL #91 |./scraped_images/mirlclub-MIRL #91.jpg |0.0349|
mirlclub-1693|MIRL #1693|./scraped_images/mirlclub-MIRL #1693.jpg|0.035 |
mirlclub-639 |MIRL #639 |./scraped_images/mirlclub-MIRL #639.jpg |0.035 |
mirlclub-462 |MIRL #462 |./scraped_images/mirlclub-MIRL #462.jpg |0.037 |
mirlclub-1545|MIRL #1545|./scraped_images/mirlclub-MIRL #1545.jpg|0.04 |
mirlclub-2104|MIRL #2104|./scraped_images/mirlclub-MIRL #2104.jpg|0.04 |
mirlclub-178 |MIRL #178 |./scraped_images/mirlclub-MIRL #178.jpg |0.04 |
mirlclub-385 |MIRL #385 |./scraped_images/mirlclub-MIRL #385.jpg |0.045 |
mirlclub-1521|MIRL #1521|./scraped_images/mirlclub-MIRL #1521.jpg|0.045 |
mirlclub-1522|MIRL #1522|./scraped_images/mirlclub-MIRL #1522.jpg|0.045 |
mirlclub-724 |MIRL #724 |./scraped_images/mirlclub-MIRL #724.jpg |0.05 |
mirlclub-2207|MIRL #2207|./scraped_images/mirlclub-MIRL #2207.jpg|0.05 |
mirlclub-628 |MIRL #628 |./scraped_images/mirlclub-MIRL #628.jpg |0.05 |
mirlclub-1899|MIRL #1899|./scraped_images/mirlclub-MIRL #1899.jpg|0.05 |
mirlclub-665 |MIRL #665 |./scraped_images/mirlclub-MIRL #665.jpg |0.055 |
mirlclub-2439|MIRL #2439|./scraped_images/mirlclub-MIRL #2439.jpg|0.06 |
mirlclub-1471|MIRL #1471|./scraped_images/mirlclub-MIRL #1471.jpg|0.06 |
mirlclub-1805|MIRL #1805|./scraped_images/mirlclub-MIRL #1805.jpg|0.065 |
mirlclub-621 |MIRL #621 |./scraped_images/mirlclub-MIRL #621.jpg |0.069 |
mirlclub-703 |MIRL #703 |./scraped_images/mirlclub-MIRL #703.jpg |0.07 |
+-------------+----------+----------------------------------------+------+
only showing top 20 rows

In [ ]:
collectionDF.toPandas().to_csv("/dbfs/mnt/data/dataset_mirlclub.csv", sep=',', header=True, index=False)

In [ ]:
collections = [
    'the-sandbox-shibuya-land-sale',
    'metroverse-genesis',
    'hapeprime',
    'collectvoxthewalkingdead',
    'decentral-games-ice',
    'jpeg-cards',
    'alphasharksofficial',
    'nyokies',
    'genesis-creepz',
    'quackyducksofficial',
    'capsulehouse',
    'coolmans-universe',
    'cyberkongz',
    'kaiju-kingz',
    'cryptoadz-by-gremplin',
    'tubby-cats',
    'deadfellaz',
    'metakages-official-collection',
    'psychedelics-anonymous-genesis',
    'town-star',
    'veecon-tickets',
    '10ktf-stockroom',
    'lvcidiaavatars',
    'neotokyo-citizens',
    'fidenza-by-tyler-hobbs',
    'g00p',
    'degentoonz-collection',
    'bossbeauties',
    'justcubesnft',
    'omni-mosquitoes-eth',
    'rtfkt-podx',
    'wolf-game',
    'thewalkingdeadofficialdarylmotorcycles',
    'bapetaverse-official',
    'mee6-avatars-pre-reveal',
    'rubberduckz',
    'nouns',
    'mv3-access-passes',
    'fluf-world',
    'chumchumsnft',
    'where-my-vans-go',
    'lazy-lions',
    'psychonautapedivision',
    'spaceridersnft',
    'bapetaverse-official',
    'the-squishiverse',
    'mirandus',
    'etherthings',
    'officialkenkyo',
    'felinefiendznft',
    'women-unite-10k-assemble',
    'chromie-squiggle-by-snowfro',
    'lilium',
    'pixel-interfaces',
    'mypethooligan',
    'shatteredeon-colonist',
    'frankfrank',
    'beari-collection',
    'crypto-unicorns-market',
    'max-pain-and-frens-by-xcopy',
    'pixel-vault-mintpass',
    'guttercatgang',
    'pjppfl',
    'fools-nft',
    'notbanksyrain',
    'metahero-generative',
    'nifty-tailor-genesis-mintpass',
    'llamaboost',
    'headtripz',
    'layer-zero-punks-eth',
    'pixels-farm',
    'dourdarcels',
    'cyberkongz-vx',
    'alphakongsclub',
    'mad-meerkat-burrow',
    'treeverse',
    'moonbirdpunks',
    'grayboys',
    'theshiboshis',
    'smiliesgenesis',
    'fvck-avatar-essence',
    'space-doodles-official',
    'pet-rock',
    'chain-runners-nft',
    'smallbros',
    'raidparty',
    'mirlclub',
    'ballies',
    'mad-hare-society-2',
    'mad-hare-society-1',
    'frenlypandas',
]


Out[78]: Column<'name'>

In [ ]:
collections = [
    'lilium',
    'pixel-interfaces',
    'mypethooligan',
    'shatteredeon-colonist',
    'frankfrank',
    'beari-collection',
    'crypto-unicorns-market',
    'max-pain-and-frens-by-xcopy',
    'pixel-vault-mintpass',
    'guttercatgang',
    'pjppfl',
    'fools-nft',
    'notbanksyrain',
    'metahero-generative',
    'nifty-tailor-genesis-mintpass',
    'llamaboost',
    'headtripz',
    'layer-zero-punks-eth',
    'pixels-farm',
    'dourdarcels',
    'cyberkongz-vx',
    'alphakongsclub',
    'mad-meerkat-burrow',
    'treeverse',
    'moonbirdpunks',
    'grayboys',
    'theshiboshis',
    'smiliesgenesis',
    'fvck-avatar-essence',
    'space-doodles-official',
    'pet-rock',
    'chain-runners-nft',
    'smallbros',
    'raidparty',
    'mirlclub',
    'ballies',
    'mad-hare-society-2',
    'mad-hare-society-1',
    'frenlypandas',
]

import os
from urllib.request import urlretrieve
from pyspark.sql.functions import *

def download_image(slug, name, url):
    try:
        urlretrieve(url, f"/dbfs/mnt/data/scraped_images/{slug}/{slug}-#{name}.png")
    except:
        print('----> Retrieve {}-#{} failed'.format(slug, name))

for col in collections:
    print('--> Running for {}'.format(col))
    filePath = '/mnt/data/collections/{}.json'.format(col)
    collection_folder = "/dbfs/mnt/data/scraped_images/{}".format(col)
    if not os.path.exists(collection_folder):
        os.makedirs(collection_folder)
    try:
        colDf = spark.read.json(filePath)
        colDf.foreach(lambda row: download_image(col, row['id'], row['image']))
    except:
        print('--> {} failed because either {} not exist or download failed'.format(col,filePath))
#     colFactor = '{}-'.format(col)
#     collectionDF = colDf.withColumn("id", concat(lit(colFactor), col('id')))
#     collectionDF = collectionDF.withColumn("path", concat(lit('./scraped_images/{}-'.format(col)), col('name'), lit('.jpg')))
#     collectionDF = collectionDF.select('id', 'name', 'path', 'price')
#     collectionDF.show(truncate=False)

--> Running for mirandus
--> mirandus failed because either /mnt/data/collections/mirandus.json not exist or download failed
--> Running for etherthings
--> Running for officialkenkyo
--> Running for felinefiendznft
--> Running for women-unite-10k-assemble
--> Running for chromie-squiggle-by-snowfro
--> chromie-squiggle-by-snowfro failed because either /mnt/data/collections/chromie-squiggle-by-snowfro.json not exist or download failed
--> Running for lilium